In [ ]:
using Neo4jBolt  
using Dates     

In [ ]:
driver = Neo4jBoltDriver("bolt://localhost:7687", auth=("neo4j", "xxxx"))

In [ ]:
mutable struct Message
    name::String
    content::String
    mod_date::Date
end

In [ ]:
   
function write_message(tx, args, kwargs)
    run(tx, 
        "MERGE (m:Message {name: \$name}) " *
        "SET m.content = \$content, m.mod_date = \$mod_date", 
        name=args[1].name, 
        content=args[1].content,
        mod_date=Dates.now()
    )
end

function read_message(tx, args, kwargs)
    result = run(tx, 
        "MATCH (m:Message {name: \$name}) " *
        "RETURN m",
        name=args[1])
    p = value(single(result)).properties
    return Message(p["name"],p["content"],p["mod_date"])
end

function link_reply(tx, args, kwargs)
     result = run(tx, 
        "MATCH (m1:Message {name: \$name1}) " *
        "MATCH (m2:Message {name: \$name2}) " *
        "MERGE (m1)-[:REPLY]->(m2) ",
        name1=args[1].name,
        name2=args[2].name
    )
end

In [ ]:
messageToWrite = Message("My message","Hi, honey, can you get me a drink, please?",Dates.now(Dates.UTC))
herReply = Message("Her reply","Of course I do, my darling. I'd do anything for you...",Dates.now(Dates.UTC))

session(driver) do sess
    write_transaction(sess, write_message, messageToWrite)
    myMessage = read_transaction(sess, read_message, "My message")
    write_transaction(sess, write_message, herReply)
    write_transaction(sess, link_reply, myMessage, herReply)
end